<a href="https://colab.research.google.com/github/kzon94/eve-marketer/blob/main/eve-marketer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests pandas

In [ ]:
import requests
import pandas as pd
import time
import os
from datetime import datetime, timezone, timedelta
from google.colab import files

INPUT_FILE = "tier_name_typeid.csv"
OUTPUT_FILE = "tier_name_price_jita.csv"

HEADERS = {
    "User-Agent": "MiCalculadoraPI/1.0 (contacto@example.com)"
}

REGION_ID = 10000002  # The Forge (Jita)
MARKET_HISTORY_ENDPOINT = "https://esi.evetech.net/latest/markets/{region_id}/history/?type_id={type_id}"
MARKET_ORDERS_ENDPOINT = "https://esi.evetech.net/latest/markets/{region_id}/orders/"
JITA_44_LOCATION_ID = 60003760

today = datetime.now(timezone.utc).date()
dates_filter = {
    "1d": today - timedelta(days=1),
    "7d": today - timedelta(days=7),
    "31d": today - timedelta(days=31)
}

def fetch_buy_price_history(type_id): # Obtiene el historial de precios de compra para un 'type_id' en The Forge
    url = MARKET_HISTORY_ENDPOINT.format(region_id=REGION_ID, type_id=type_id)

    try:
        resp = requests.get(url, headers=HEADERS, timeout=10)
        resp.raise_for_status()
        history = resp.json()

        if not history:
            return None

        df = pd.DataFrame(history)
        df['date'] = pd.to_datetime(df['date']).dt.date
        results = {}

        for period, min_date in dates_filter.items():
            df_filtered = df[df['date'] >= min_date]
            if df_filtered.empty:
                results[f"avg_buy_price_{period}"] = None
                results[f"max_buy_price_{period}"] = None
                results[f"min_buy_price_{period}"] = None
                results[f"buy_volume_{period}"] = None
            else:
                results[f"avg_buy_price_{period}"] = round(df_filtered['average'].mean(), 2)
                results[f"max_buy_price_{period}"] = df_filtered['highest'].max()
                results[f"min_buy_price_{period}"] = df_filtered['lowest'].min()
                results[f"buy_volume_{period}"] = df_filtered['volume'].sum()

        return results

    except requests.exceptions.RequestException as e:
        print(f"Error al obtener datos de mercado para type_id={type_id}: {e}")
        return None

def fetch_current_buy_price(type_id): # Obtiene el precio máximo de compra actual en Jita 4-4 para un type_id
    params = {"datasource": "tranquility", "order_type": "buy", "type_id": type_id}

    try:
        resp = requests.get(MARKET_ORDERS_ENDPOINT.format(region_id=REGION_ID), headers=HEADERS, params=params, timeout=10)
        resp.raise_for_status()
        orders = resp.json()

        buy_orders = [o['price'] for o in orders if o.get('is_buy_order') and o.get('location_id') == JITA_44_LOCATION_ID]

        return max(buy_orders) if buy_orders else None

    except requests.exceptions.RequestException as e:
        print(f"Error al obtener precio actual para type_id={type_id}: {e}")
        return None

def main():
    try:
        df_in = pd.read_csv(INPUT_FILE)
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo {INPUT_FILE}. Por favor, súbelo a la sesión de Colab.")
        return

    required_columns = {'tier', 'name', 'type_id'}
    if not required_columns.issubset(df_in.columns):
        print("Error: El CSV de entrada debe tener 'tier', 'name' y 'type_id'.")
        return

    type_ids = df_in[['tier', 'name', 'type_id']].to_dict('records')
    results = []
    batch_size = 20

    for i in range(0, len(type_ids), batch_size):
        batch = type_ids[i : i+batch_size]
        for item in batch:
            tid = item['type_id']
            name = item['name']
            tier = item['tier']

            current_price = fetch_current_buy_price(tid)
            price_data = fetch_buy_price_history(tid)

            if price_data:
                results.append({"tier": tier, "name": name, "type_id": tid, "current_buy_price": current_price, **price_data})
        time.sleep(1)  # Evitar sobrecarga en la API

    df_out = pd.DataFrame(results)
    df_out.to_csv(OUTPUT_FILE, index=False)
    print(f"Archivo generado: {OUTPUT_FILE}")

    # Descargar archivo al sistema local en Colab
    files.download(OUTPUT_FILE)

if __name__ == "__main__":
    main()